In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, when, first, avg
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from functools import reduce
from pyspark.sql import DataFrame

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# Carregar tabelas
time_df = spark.table("database.time")
wage_df = spark.table("database.wage")

# Lista de países e anos
paises = ["%Germany%", "%Switzerland%", "%Poland%", "%Hungary%"]
anos = ["2010", "2013"]

# Filtrando os DataFrames

def filter_by_multiple_conditions(df: DataFrame, paises: list, anos: list) -> DataFrame:
    conditions = reduce(lambda a, b: a | b, (col("nome_pais").like(p) for p in paises))
    df = df.filter(conditions)
    df = df.filter(col("ano").isin(anos))
    return df

time_df = filter_by_multiple_conditions(time_df, paises, anos)
wage_df = filter_by_multiple_conditions(wage_df, paises, anos)

time_df = time_df.drop("cod_indicador", "cod_pais")

wage_df = wage_df.drop("cod_indicador", "cod_pais")

wage_df = wage_df.withColumnRenamed('Disaggregation', 'desagregacao')

wage_df = wage_df.filter((col('desagregacao') == 'female') | (col('desagregacao') == 'male'))

# Filtrar pelos indicadores de emprego
wage_df = wage_df.filter(F.col("nome_indicador").like("%Wage and salaried workers%"))

wage_df = wage_df.filter(F.col("desagregacao").isNotNull())

# Calcular a média dos valores por ano, gênero e indicador
wage_df_media = wage_df.groupBy("nome_pais", "ano", "desagregacao", "nome_indicador") \
    .agg(F.avg("valor").alias("valor"))

# Selecionar apenas uma linha por ano e gênero
window = Window.partitionBy("nome_pais", "ano", "desagregacao").orderBy(F.col("valor"))

wage_df_final = wage_df_media.withColumn("row_number", F.row_number().over(window)) \
    .filter(F.col("row_number") == 1).drop("row_number")

# Definindo a ordem das colunas em 
columns_order = ["nome_indicador", "nome_pais", "ano", "valor", "desagregacao"]

# Reorganizando as colunas no DataFrame 
wage_df_final = wage_df_final.select(columns_order)

In [3]:
time_df = time_df.withColumn("valor", col("valor").cast("float"))
wage_df_final = wage_df_final.withColumn("valor", col("valor").cast("float"))

In [4]:
df_concatenado = wage_df_final.union(time_df)
df_concatenado = df_concatenado.withColumn("ano", col("ano").cast("int"))

In [5]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

df_concatenado.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaWageTimeGold")

In [6]:
spark.sql(
    """
    SELECT * FROM database.WageTimeGold
    """
).show(500)

+-----------+--------------------+----+--------+------------+
|  nome_pais|      nome_indicador| ano|   valor|desagregacao|
+-----------+--------------------+----+--------+------------+
|    Germany|Wage and salaried...|2010|91.57062|      female|
|    Germany|Wage and salaried...|2010|85.74215|        male|
|    Germany|Wage and salaried...|2013|91.88864|      female|
|    Germany|Wage and salaried...|2013|86.12325|        male|
|    Hungary|Wage and salaried...|2010|91.06637|      female|
|    Hungary|Wage and salaried...|2010|84.73759|        male|
|    Hungary|Wage and salaried...|2013|91.76857|      female|
|    Hungary|Wage and salaried...|2013|86.13021|        male|
|     Poland|Wage and salaried...|2010|79.91518|      female|
|     Poland|Wage and salaried...|2010|74.68536|        male|
|     Poland|Wage and salaried...|2013|81.56346|      female|
|     Poland|Wage and salaried...|2013|75.45581|        male|
|Switzerland|Wage and salaried...|2010|86.87634|      female|
|Switzer

In [7]:
spark.stop()